In [1]:
# Scientific libraries
              
import numpy as np              
import scipy as sp              
import pandas as pd              
import json              
import missingno as msno              

# Loading Plotting Modules
              
import matplotlib              
import matplotlib.pyplot as plt              
import seaborn as sns              
%matplotlib inline              
import chart_studio.plotly as py              
import plotly.figure_factory as ff              
import plotly.graph_objects as go              
import plotly.express as px              
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot              
init_notebook_mode(connected=True)              

# Setting Data Frame Options
              
pd.set_option('display.max_rows', 40)              
pd.set_option('display.max_columns', 50)              
pd.set_option('display.width', 100)              
pd.set_option('display.max_colwidth', 100)              

# Setting Plot Configuration
              
sns.set(rc={'figure.figsize':(19,11)}, style = 'white')              

import katana.nbutil as ktn
              
import os

In [2]:
num_data = pd.read_parquet('./output/pre-modeling/all_num_processed.pqt.gz')
cat_data = pd.read_parquet('./output/pre-modeling/r_reduced_all_cat.pqt.gz')
print('Data Frame (num): %d x %d' % num_data.shape)
print('Data Frame (cat): %d x %d' % cat_data.shape)

Data Frame (num): 1097231 x 382
Data Frame (num): 1097231 x 418


In [3]:
indexes = pd.read_csv('./output/transaction_ids.csv.gz')

In [5]:
merged_data = num_data.join(cat_data, how='left')
del num_data, cat_data
gc.collect()

7642

In [6]:
merged_data.head()

,C1,C10,C11,C12,C13,C14,C2,C3,C4,C5,C6,C7,C8,C9,TransactionAmt,dist1,dist2,V263,V217,V110,V188,V154,V220,V332,V174,...,id_33_nan,id_34_match_status:1,id_34_match_status:2,id_34_nan,id_35_F,id_35_T,id_35_nan,id_36_F,id_36_T,id_36_nan,id_37_F,id_37_T,id_37_nan,id_38_F,id_38_T,id_38_nan,DeviceType_desktop,DeviceType_mobile,DeviceType_nan,DeviceInfo_MacOS,DeviceInfo_Trident/7.0,DeviceInfo_Windows,DeviceInfo_iOS Device,DeviceInfo_rv:11.0,DeviceInfo_nan
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987000,0.000000,NaN,0.693147,NaN,0.000000,0.0,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,0.0,4.226834,2.944439,NaN,NaN,NaN,-0.087282,NaN,NaN,NaN,NaN,NaN,...,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1
2987001,0.000000,NaN,0.000000,NaN,0.000000,0.0,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,3.367296,NaN,NaN,NaN,NaN,-0.087282,NaN,NaN,NaN,NaN,NaN,...,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1
2987002,0.000000,NaN,0.000000,NaN,0.000000,0.0,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,0.0,4.077537,5.659482,NaN,NaN,NaN,-0.087282,NaN,NaN,NaN,NaN,NaN,...,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1
2987003,0.693147,NaN,0.000000,NaN,3.218876,0.0,1.609438,NaN,NaN,NaN,1.386294,NaN,NaN,0.0,3.912023,NaN,NaN,NaN,NaN,-0.087282,NaN,NaN,NaN,NaN,NaN,...,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1
2987004,0.000000,0.0,0.000000,NaN,0.000000,0.0,0.000000,NaN,NaN,NaN,0.000000,NaN,0.0,NaN,3.912023,NaN,NaN,-0.032664,-0.124815,-0.087282,-0.007128,-1.458688,-0.137183,-0.084668,-0.33739,...,0,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0


In [7]:
nums = pd.read_parquet('./output/pre-modeling/all_num_data.pqt.gz')

In [8]:
nums = nums[['isFraud']]

In [9]:
merged_data = merged_data.join(nums, how='left')

In [10]:
del nums
gc.collect()

1182

In [ ]:
merged_data.to_parquet('./output/pre-modeling/merged_data_processed.pqt.gz')

In [20]:
train_idx = pd.Series(idx[idx.train_set_id.notna()].train_set_id.tolist() \
                      + idx[idx.dev_set_id.notna()].dev_set_id.tolist(), dtype=np.int32).tolist()
test_idx = pd.Series(idx.test_set_id, dtype=np.int32).tolist()

In [23]:
train_md = merged_data.loc[train_idx]
test_md = merged_data.loc[test_idx]
del merged_data
gc.collect()

13774

In [25]:
train_md.to_parquet('../modeling/model_data/reduced_processed/train_set.pqt.gz')
test_md.to_parquet('../modeling/model_data/reduced_processed/test_set.pqt.gz')